In [ ]:
! pip install xgboost
! pip install pandas
! pip install scikit-learn
! pip install pyarrow

In [ ]:
from pyspark.ml import Pipeline, PipelineModel
from Read_In_Data.read_data import Reading_Data
import os


################################ Read In Modules ################################
reading_data=Reading_Data()


################################ Read In Data ################################
# Testing Summary Stats Data
test_files=list(map(lambda x: os.path.join(os.path.abspath('/cephfs/summary_stats/all_test_bool_updated'), x),
                                           os.listdir('/cephfs/summary_stats/all_test_bool_updated')))
test_files=[i for i in test_files if not ('.crc' in i or 'SUCCESS' in i)]

# Calling DataFrames
summary_stats_test=reading_data.read_in_all_summary_stats(file_list=test_files)
summary_stats_test.show(2)
print((summary_stats_test.count(), len(summary_stats_test.columns)))

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
pipelineModel_class=PipelineModel.load("/cephfs/Saved_Models/Hyperparameter_Tuning/XGBClassification")
feature_importance_dict=pipelineModel_class.stages[-1].bestModel.get_booster().get_score(importance_type="gain")

feature_names=pipelineModel_class.stages[-2].extractParamMap()[list(pipelineModel_class.stages[-2].\
                                                              extractParamMap().keys())[-1]]
feature_importance_dict=dict(zip(feature_names, list(feature_importance_dict.values())))
feature_importance_df=pd.DataFrame(feature_importance_dict, index=[0])

feature_importance_df=feature_importance_df.transpose().\
                      reset_index(drop=False).\
                      rename(columns={'index': 'Feature', 0: 'Accuracy Gain'}).\
                      sort_values(by='Accuracy Gain', ascending=False).\
                      reset_index(drop=True)

In [ ]:
feature_importance_df=feature_importance_df.sort_values(by='Accuracy Gain', ascending=True)
fig = px.bar(feature_importance_df, 
             x='Accuracy Gain', 
             y='Feature', 
             orientation='h', 
             color='Accuracy Gain',
             height=1000,
             width=900,
             color_continuous_scale='YlGn')
fig.write_image('/home/jovyan/glucose-data-analysis/Output_Files/Classification/xgboost_classification_feature_importance.png')